<a href="https://colab.research.google.com/github/isaac-lusuku/GROUP25_REPO/blob/main/final_animal_body_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


importing the necessaries libraries

In [2]:
import os
import pandas as pd
from glob import glob
import tensorflow as tf
from tensorflow.keras import losses, optimizers, layers, Sequential, callbacks
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers

Get image data

In [3]:
os.makedirs('body_score_dataset', exist_ok=True)
os.system('git clone -b body_scores_prediction_dataset https://github.com/MVet-Platform/M-Vet_Hackathon24.git ./body_score_dataset')

0

Load label data

In [4]:
df_train_data = pd.read_csv('body_score_dataset/train_data.csv')

Get file path for image files

In [5]:
df_train_data['filepath'] = df_train_data.apply(lambda row: glob(f'body_score_dataset/**/{row.filename}')[0], axis=1)

Create array of body scores and file paths

In [6]:
body_scores = df_train_data.bodyScore.values
file_paths = df_train_data.filepath.values

preprocessing function

In [23]:
def load_and_preprocess_image(file_path, body_score=None):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    if body_score is not None:
        return image, body_score
    else:
        return image

Create a TensorFlow dataset

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((file_paths, body_scores))
dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

sample image display

In [ ]:
for item in dataset.take(1):
    plt.title(f'Body Score: {item[1].numpy()}')
    plt.imshow(item[0].numpy())
    plt.axis('off')
    plt.show()


Shuffle and split the dataset

In [10]:
dataset = dataset.shuffle(buffer_size=2000)
train_size = int(0.7 * len(file_paths))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

Batch the datasets

In [11]:
train_dataset = train_dataset.batch(batch_size=32)
val_dataset = val_dataset.batch(batch_size=32)

In [12]:
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

model = Sequential()
model.add(base_model)

# CONV LAYER ONE
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())

# CONV LAYER TWO
model.add(layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.BatchNormalization())

# CONV LAYER THREE
# model.add(layers.Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

# CONV LAYER FOUR
# model.add(layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

# GLOBAL POOLING LAYER
model.add(layers.GlobalAveragePooling2D())

# DENSE LAYER ONE
model.add(layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(layers.Dropout(0.5))

# DENSE LAYER TWO
# model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
# model.add(layers.Dropout(0.5))

# FINAL LAYER
model.add(layers.Dense(units=1, activation='linear'))


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Compile model

In [13]:
model.compile(loss=losses.mae, optimizer=optimizers.RMSprop())

Callbacks

In [14]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

model_checkpoint = callbacks.ModelCheckpoint(
    'best_model.keras',  # Change the filename to end with '.keras'
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False
)

Train model with callbacks

In [15]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=20,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 164s 628ms/step - loss: 2.6601 - val_loss: 10.2312
Epoch 2/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 78s 442ms/step - loss: 1.0696 - val_loss: 2.4470
Epoch 3/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 84s 449ms/step - loss: 0.8287 - val_loss: 0.9843
Epoch 4/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 76s 420ms/step - loss: 0.7458 - val_loss: 1.3277
Epoch 5/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 91s 474ms/step - loss: 0.7567 - val_loss: 0.7011
Epoch 6/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 132s 416ms/step - loss: 0.7194 - val_loss: 12.8575
Epoch 7/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 89s 499ms/step - loss: 0.7274 - val_loss: 0.7003
Epoch 8/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 143s 515ms/step - loss: 0.7187 - val_loss: 493.5393
Epoch 9/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 90s 516ms/step - loss: 0.6971 - val_loss: 4.1523
Epoch 10/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 85s 483ms/step - loss: 0.7154 - val_loss: 0.6651
Epoch 11/20
167/167 ━━━━━━━━━━━━━━━━━━━━ 132s 416ms/step - loss: 0.7105 - val_loss: 2.3677
Epoch 12/20

Visualize training statistics

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

if 'accuracy' in history.history:
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

plt.show()

Evaluate the model on the validation dataset

In [22]:
val_loss = model.evaluate(val_dataset)
print(f'Validation Loss: {val_loss}')

72/72 ━━━━━━━━━━━━━━━━━━━━ 28s 86ms/step - loss: 0.6564
Validation Loss: 0.6596803665161133


saving the model

In [20]:
model.save('model.h5')


downloading the model

In [21]:
from google.colab import files
files.download('model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>